This notebook is discussed in the below discussion:

- https://www.kaggle.com/competitions/birdclef-2022/discussion/323582

In [ ]:
!pip install nb-black >/dev/null

In [ ]:
import warnings
from types import SimpleNamespace

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import librosa
import librosa.display
import IPython.display as ipd

warnings.filterwarnings("ignore")
plt.style.use("ggplot")

%load_ext lab_black

In [ ]:
cfg = SimpleNamespace(
    sample_rate=32_000,
    window_size=2048,
    hop_length=512,
    mel_bins=128,
    fmin=0,
    fmax=16_000,
)

In [ ]:
def create_call_meta_df(
    call_df, meta_df, train_meta_df, unit_frame_sec=5, sample_rate=32_000
):
    call_df = call_df.copy()
    meta_df = meta_df.copy()
    train_meta_df = train_meta_df.copy()
    meta_df["num_5_sec_frames"] = meta_df.length // (unit_frame_sec * sample_rate) + 1
    meta_df["last_end_time"] = meta_df["num_5_sec_frames"] * unit_frame_sec

    call_meta_df = pd.merge(call_df, meta_df, on="filename", how="left")
    call_meta_df = call_meta_df.query("end_time <= last_end_time")

    train_meta_df.rename({"filename": "original_filename"}, axis=1, inplace=True)
    call_meta_df = pd.merge(
        call_meta_df,
        train_meta_df,
        on="original_filename",
        how="left",
    )
    return call_meta_df


def split_pos_neg(input_df):
    input_df = input_df.copy()
    pos, neg = input_df.query("target > 0.5"), input_df.query("target <= 0.5")
    return pos, neg


def visualize(rel_path, input_df, title):
    input_df = input_df.copy()
    path = f"../input/birdclef-2022-subclip-60-sec/train_audio/{rel_path}"
    display(ipd.Audio(path))

    # show mel spec
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(24, 6), sharex=True)
    audio, _ = librosa.core.load(path, sr=cfg.sample_rate, mono=True)
    melspec = librosa.feature.melspectrogram(
        audio,
        sr=cfg.sample_rate,
        n_fft=cfg.window_size,
        hop_length=cfg.hop_length,
        n_mels=cfg.mel_bins,
        power=1.0,
        fmin=cfg.fmin,
        fmax=cfg.fmax,
    )
    spec = librosa.pcen(
        melspec * (2**31),
        time_constant=0.06,
        eps=1e-6,
        gain=0.8,
        power=0.25,
        bias=10,
        sr=cfg.sample_rate,
        hop_length=cfg.hop_length,
    )
    colormesh = librosa.display.specshow(
        spec,
        hop_length=cfg.hop_length,
        sr=cfg.sample_rate,
        fmin=cfg.fmin,
        fmax=cfg.fmax,
        x_axis="time",
        y_axis="mel",
        ax=ax1,
    )
    ax1.set_title(
        title,
        fontsize=15,
    )

    ax2 = sns.lineplot(
        x="time_sec",
        y="target",
        data=input_df.query("filename == @rel_path"),
        marker="o",
    )
    ax2.set(ylim=(0, 1))

# Sub clip 60 sec

In [ ]:
call_df = pd.read_csv(
    "../input/birdclef22-add-binary-classifier-label-60sec/birdclef_2022_call_meta.csv"
)
meta_df = pd.read_csv("../input/birdclef-2022-subclip-60-sec/subclip_meta.csv")
train_meta_df = pd.read_csv("../input/birdclef-2022/train_metadata.csv")
scored_birds = pd.read_json("../input/birdclef-2022/scored_birds.json")[0].tolist()

In [ ]:
call_meta_df = create_call_meta_df(call_df, meta_df, train_meta_df)

In [ ]:
len(call_meta_df.query("target > 0.5")) / len(call_meta_df)

In [ ]:
_, ax = plt.subplots()
sns.histplot(call_meta_df.target, ax=ax)
ax.set(title="Distribution of call probability")
plt.show()

In [ ]:
scored_df = train_meta_df.query("primary_label in @scored_birds")
sorted_scored_birds = scored_df.value_counts("primary_label").keys().tolist()

In [ ]:
pos_paths = []
for bird in sorted_scored_birds:
    pos_path = (
        call_meta_df.query("primary_label == @bird and target > 0.5")
        .sample(1)
        .filename.item()
    )
    pos_paths.append(pos_path)

In [ ]:
for i, p_path in enumerate(pos_paths):
    print(f"{i+1}: {p_path}")
    visualize(p_path, call_meta_df, title=p_path)
    plt.show()

# subclip 5 sec

In [ ]:
call_df_5sec = pd.read_csv(
    "../input/birdclef22-add-binary-classifier-label-5sec/birdclef_2022_call_meta_5_sec.csv"
)

In [ ]:
call_meta_df_5sec = create_call_meta_df(call_df_5sec, meta_df, train_meta_df)

In [ ]:
len(call_meta_df_5sec.query("target > 0.5")) / len(call_meta_df_5sec)

In [ ]:
_, ax = plt.subplots()
sns.histplot(call_meta_df_5sec.target, ax=ax)
ax.set(title="Distribution of call probability")
plt.show()